In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = value_counts[value_counts < 528].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
value_counts2 = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = value_counts2[value_counts2 < 1883].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df,dtype=float)
dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization 1: adding neurons

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn1 = tf.keras.models.Sequential()

# First hidden layer
# `X_train_scaled.shape[1]` returns the number of columns in the training data,
# which represents the number of features the model needs to expect in the input layer.
nn1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim = X_train_scaled.shape[1]))

# Second hidden layer
#add more units
nn1.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10,081
Trainable params: 10,081
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Train the model
fit_model = nn1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5713 - accuracy: 0.7236
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5566 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7255 - 313ms/epoch - 1ms/step
Loss: 0.5594645738601685, Accuracy: 0.7254810333251953


In [41]:
# Export our model to HDF5 file
from google.colab import files

nn1.save('AlphabetSoupCharity_Optimization1.h5')
files.download('AlphabetSoupCharity_Optimization1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Optimization 2: adding another layer

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
# `X_train_scaled.shape[1]` returns the number of columns in the training data,
# which represents the number of features the model needs to expect in the input layer.
nn2.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim = X_train_scaled.shape[1]))

# Second hidden layer
#add more units
nn2.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Add a hidden layer
nn2.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 80)                6480      
                                                                 
 dense_8 (Dense)             (None, 80)                6480      
                                                                 
 dense_9 (Dense)             (None, 1)                 81        
                                                                 
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5691 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7346
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7357
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7353
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7366
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5781 - accuracy: 0.7255 - 285ms/epoch - 1ms/step
Loss: 0.5780602693557739, Accuracy: 0.7254810333251953


In [40]:
# Export our model to HDF5 file
from google.colab import files

nn2.save('AlphabetSoupCharity_Optimization2.h5')
files.download('AlphabetSoupCharity_Optimization2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Optimization 3: change activation to tanh and epochs to 200

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()

# First hidden layer
# `X_train_scaled.shape[1]` returns the number of columns in the training data,
# which represents the number of features the model needs to expect in the input layer.
nn3.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim = X_train_scaled.shape[1]))

# Second hidden layer
#add more units
nn3.add(tf.keras.layers.Dense(units=80, activation="tanh"))

# Add a hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 80)                3520      
                                                                 
 dense_31 (Dense)            (None, 80)                6480      
                                                                 
 dense_32 (Dense)            (None, 80)                6480      
                                                                 
 dense_33 (Dense)            (None, 1)                 81        
                                                                 
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [34]:
nn3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn3.fit(X_train_scaled, y_train, epochs=200)


Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5706 - accuracy: 0.7206
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5581 - accuracy: 0.7268
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5555 - accuracy: 0.7288
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7298
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7308
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7316
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7325
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7341
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7333
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accura

In [35]:
model_loss, model_accuracy = nn3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5583 - accuracy: 0.7258 - 590ms/epoch - 2ms/step
Loss: 0.5582984089851379, Accuracy: 0.7258309125900269


In [39]:
# Export our model to HDF5 file
from google.colab import files

nn3.save('AlphabetSoupCharity_Optimization3.h5')
files.download('AlphabetSoupCharity_Optimization3.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>